In [1]:
import numpy as np
from PIL import Image
from pathlib import Path

استدعاء مجلد الصور و معرفة عدد مجلدات و الملفات داخلة:

In [2]:
file= Path(r"C:\Users\Farah Alfarjani\super-class-img\data")
#__
folder=[fd for fd in file.glob('**/*') if fd.is_dir()]
print('عدد المجلدات :',len(folder))
print("اسماء المجلدات:")
for fd in folder:
 print(fd.name)
#__
files=[f for f in file.glob('**/*') if f.is_file()]
print('عدد الملفات داخل المجلدات:',len(files))

عدد المجلدات : 3
اسماء المجلدات:
covid
normal
virus
عدد الملفات داخل المجلدات: 1823


معرفة عدد الصور الذي توجد داخل كل مجلد

In [3]:
normal_path=file/"normal"
normal_file=[f for f in normal_path.glob('**/*') if f.is_file()]
print("عدد الملفات داخل مجلد normal:",len(normal_file))
#__
covid_path=file/'covid'
covid_file=[f for f in covid_path.glob('**/*') if f.is_file()]
print("عدد الملفات داخل مجلد covid:",len(covid_file))
#___
virus_path=file/'virus'
virus_file=[f for f in virus_path.glob('**/*') if f.is_file()]
print("عدد الملفات داخل مجلد virus:",len(virus_file))

عدد الملفات داخل مجلد normal: 668
عدد الملفات داخل مجلد covid: 536
عدد الملفات داخل مجلد virus: 619


# وتحميل الصور من المجلدات و ربط كل فئة بتصنيف و معالجة الصور

In [4]:
#من copilot
# افتحي صورة واحدة للتجربة
f = files[55]   # أول ملف من القائمة
img = Image.open(f)

# اطبعي معلومات الصورة الأصلية
print("اسم الملف:", f.name)
print("وضع الألوان (mode):", img.mode)   # مثلاً RGB أو RGBA أو L
print("حجم الصورة الأصلي:", img.size)    # (العرض, الطول)

# حولي الصورة إلى مصفوفة NumPy بدون أي تغيير
arr_original = np.array(img)
print("شكل المصفوفة الأصلية:", arr_original.shape)
img.show()

اسم الملف: 8FDE8DBA-CFBD-4B4C-B1A4-6F36A93B7E87.jpeg
وضع الألوان (mode): RGB
حجم الصورة الأصلي: (657, 657)
شكل المصفوفة الأصلية: (657, 657, 3)


In [9]:
label_name = {"covid":0, "normal":1, "virus":2}
data=[]
labels=[]
img_path={".png", ".jpg", ".jpeg", ".jfif", ".L"} #امتدادت الصور في الملفات

for file_name, label in label_name.items():
    file_name_path=file/file_name
    files=[ f for f in file_name_path.glob('**/*') if f.is_file() and f.suffix.lower() in img_path]
#suffixلامتداد الملف مع النقطة و lower تحويل حروف الامتداد الى صغيرة 
    print((file_name), len(files))
    for f in files:
        img=Image.open(f).convert("L").resize((512,512))
        arr= np.array(img, dtype=np.float32)/255.0
        data.append(arr.flatten())
        labels.append(label)
print("عدد الصور:", len(data))
print("عدد التصنيفات:", len(labels))

covid 536
normal 668
virus 619
عدد الصور: 1823
عدد التصنيفات: 1823


In [10]:
X = np.array(data, dtype=np.float32)
y = np.array(labels, dtype=np.int64)

print("شكل X:", X.shape)   # (عدد الصور, حجم المتجه)
print("شكل y:", y.shape)   # (عدد الصور,)

شكل X: (1823, 262144)
شكل y: (1823,)


In [7]:
# افتحي صورة واحدة للتجربة
f = files[55]   # أول ملف من القائمة
img = Image.open(f)

# اطبعي معلومات الصورة الأصلية
print(" اسم الملف:", f.name)
print("وضع الألوان (mode):", img.mode)   # مثلاً RGB أو RGBA أو L
print("حجم الصورة الأصلي:", img.size)    # (العرض, الطول)

# حولي الصورة إلى مصفوفة NumPy بدون أي تغيير
arry = np.array(img)
print("شكل المصفوفة بعد التعديل:", arr.shape)
img.show()

 اسم الملف: person1214_virus_2059.jpeg
وضع الألوان (mode): L
حجم الصورة الأصلي: (1160, 856)
شكل المصفوفة بعد التعديل: (512, 512)


In [11]:
# تقسيم البيانات
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
#______
# تقليل الأبعاد باستخدام PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=100, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
#______
# تدريب نموذج 
from sklearn.svm import SVC
model = SVC(kernel="rbf", probability=True, random_state=42)
model.fit(X_train_pca, y_train)
# تقييم النموذج
y_pred = model.predict(X_test_pca)
#____
from sklearn.metrics import classification_report
print("تقرير التصنيف:\n", classification_report(y_test, y_pred))
#_____
# حفظ النماذج
import joblib
joblib.dump(pca, "pca_model.joblib")
joblib.dump(model, "svm_model.joblib")

تقرير التصنيف:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96       161
           1       0.92      0.90      0.91       200
           2       0.90      0.89      0.89       186

    accuracy                           0.92       547
   macro avg       0.92      0.92      0.92       547
weighted avg       0.92      0.92      0.92       547



['svm_model.joblib']